# 使用RNN生成金庸体文本

我们使用金庸先生的小说作为数据源，尝试让机器也能写出金庸风格的小说。

参考：
- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) 
- [eager execution](https://www.tensorflow.org/guide/eager)

## Setup

### 导入 TensorFlow 和其他库

In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### 下载金庸小说数据集

In [4]:
path_to_file = tf.keras.utils.get_file('tianlongbabu.txt', 'http://aimaksen.bslience.cn/jinyong/%E5%A4%A9%E9%BE%99%E5%85%AB%E9%83%A8.txt')

3825664/3810759 [==============================] - 3s 1us/step


### 读取数据

首先，我们先探索一下数据

In [7]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'文本总长度: {len(text)} 字')

文本总长度: 1278131 字


In [8]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿释名
　　“天龙八部”这名词出于佛经。许多大乘佛经叙述佛陀向诸菩萨、比丘等说法时，常有天龙八部参与听法。如《法华经·提婆达多品》：“天龙八部、人与非人，皆遥见彼龙女成佛”。“非人”是形貌似人而实际不是人的众生。“天龙八部”都是“非人”，包括八种神道怪物，因为以“天”及“龙”为首，所以称为“天龙八部”。八部者，一天，二龙，三夜叉，四乾达婆，五阿修罗，六迦楼罗，七紧那罗，八摩呼罗迦。
　　“天”是指天神。在佛教中，天神的地位并非至高无上，只不过比人能享受到更大、更长久的福报而已。佛教认为一切事物无


In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} 个不重复的字')

4268 个不重复的字


## 处理文本

### 将文本转换为向量

在训练之前，我们需要把文本转换为一种数字化的表述。

`preprocessing.StringLookup` 层可以帮助我们把每一个字转换为相应的数字 ID。只需要我们先把文本转化为一个个的 token。 

In [12]:
example_texts = ['天龙八部', '金庸']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'\xe5\xa4\xa9', b'\xe9\xbe\x99', b'\xe5\x85\xab', b'\xe9\x83\xa8'], [b'\xe9\x87\x91', b'\xe5\xba\xb8']]>

现在我们可以创建一个 `preprocessing.StringLookup` 层:

In [13]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

它可以把 token 都转化为相应的 ID:

In [14]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[907, 4257, 344, 3824], [3857, 1255]]>

由于，我们的目的是为了生成文本，所以，在我们生成的过程中也需要把 ID 转换成文本的操作，我们可以使用 `preprocessing.StringLookup(..., invert=True)` 来达成这个目的。

注意：这里我们使用 `get_vocabulary()` 来代替原来使用的 `sorted(set(text))`  获取字典，这种方法会把我们再生成过程中产生的 `[UNK]` token 也包含进去。

https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization#get_vocabulary

In [15]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

这层会把生成的 token ID 转换为 `tf.RaggedTensor` 格式的相应的文字。

In [16]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'\xe5\xa4\xa9', b'\xe9\xbe\x99', b'\xe5\x85\xab', b'\xe9\x83\xa8'], [b'\xe9\x87\x91', b'\xe5\xba\xb8']]>


我们可以使用 `tf.strings.reduce_join` 把这些字重新拼接成为文本。

In [17]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'\xe5\xa4\xa9\xe9\xbe\x99\xe5\x85\xab\xe9\x83\xa8',
       b'\xe9\x87\x91\xe5\xba\xb8'], dtype=object)

In [19]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### 生成任务（预测任务）

给定一个字，或者是一串字，哪个字可能是接下来会出现的额可能性最大的字？

这个就是我们要做的模型。我们使用 RNN 模型来读取前面出现的字，并且把他们转换成一个内部的状态，根据这个内部的状态，我们去生成（预测）下一个字是什么？

### 创建训练样本和目标(target)

接下来，我们要把文本都切分成样本序列。 每个输入的样本序列，都会包含 `seq_length` 个字(token)。

对于每一个输入的序列，对应的目标(target)包含了和输入序列相同的长度，只是会向右侧移动一位。

我们会把所有的文本都切割成 `seq_length+1` 的长度.比如说， `seq_length` 是 4，并且我们的文本是 "我是中国人". 输入的文本会是 "我是中国",  target 序列是 "是中国人".

为了能够把数据转换成上述的格式，我们使用 `tf.data.Dataset.from_tensor_slices` 方法把文本向量转换成一些列的 token 序列切片(slice)。

In [20]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1278131,), dtype=int64, numpy=array([4261, 3852,  596, ...,    1,    1,    1])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [22]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

﻿
释
名


　
　
“
天
龙
八


In [23]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

`batch` 方法可以让我们把这些文本转换我们期望长度的序列。

In [25]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'\xef\xbb\xbf' b'\xe9\x87\x8a' b'\xe5\x90\x8d' b'\n' b'\xe3\x80\x80'
 b'\xe3\x80\x80' b'\xe2\x80\x9c' b'\xe5\xa4\xa9' b'\xe9\xbe\x99'
 b'\xe5\x85\xab' b'\xe9\x83\xa8' b'\xe2\x80\x9d' b'\xe8\xbf\x99'
 b'\xe5\x90\x8d' b'\xe8\xaf\x8d' b'\xe5\x87\xba' b'\xe4\xba\x8e'
 b'\xe4\xbd\x9b' b'\xe7\xbb\x8f' b'\xe3\x80\x82' b'\xe8\xae\xb8'
 b'\xe5\xa4\x9a' b'\xe5\xa4\xa7' b'\xe4\xb9\x98' b'\xe4\xbd\x9b'
 b'\xe7\xbb\x8f' b'\xe5\x8f\x99' b'\xe8\xbf\xb0' b'\xe4\xbd\x9b'
 b'\xe9\x99\x80' b'\xe5\x90\x91' b'\xe8\xaf\xb8' b'\xe8\x8f\xa9'
 b'\xe8\x90\xa8' b'\xe3\x80\x81' b'\xe6\xaf\x94' b'\xe4\xb8\x98'
 b'\xe7\xad\x89' b'\xe8\xaf\xb4' b'\xe6\xb3\x95' b'\xe6\x97\xb6'
 b'\xef\xbc\x8c' b'\xe5\xb8\xb8' b'\xe6\x9c\x89' b'\xe5\xa4\xa9'
 b'\xe9\xbe\x99' b'\xe5\x85\xab' b'\xe9\x83\xa8' b'\xe5\x8f\x82'
 b'\xe4\xb8\x8e' b'\xe5\x90\xac' b'\xe6\xb3\x95' b'\xe3\x80\x82'
 b'\xe5\xa6\x82' b'\xe3\x80\x8a' b'\xe6\xb3\x95' b'\xe5\x8d\x8e'
 b'\xe7\xbb\x8f' b'\xc2\xb7' b'\xe6\x8f\x90' b'\xe5\xa9\x86'
 b'\xe8\xbe\

当我们把👆🏻的转换为文本，就可以非常清晰的看到它在做什么。

In [31]:
for seq in sequences.take(5):
    print(str(text_from_ids(seq).numpy(), encoding='utf-8'))
    print('-----------')

﻿释名
　　“天龙八部”这名词出于佛经。许多大乘佛经叙述佛陀向诸菩萨、比丘等说法时，常有天龙八部参与听法。如《法华经·提婆达多品》：“天龙八部、人与非人，皆遥见彼龙女成佛”。“非人”是形貌似人而实际不是
-----------
人的众生。“天龙八部”都是“非人”，包括八种神道怪物，因为以“天”及“龙”为首，所以称为“天龙八部”。八部者，一天，二龙，三夜叉，四乾达婆，五阿修罗，六迦楼罗，七紧那罗，八摩呼罗迦。
　　“天”是指天神
-----------
。在佛教中，天神的地位并非至高无上，只不过比人能享受到更大、更长久的福报而已。佛教认为一切事物无常，天神的寿命终了之后，也是要死的。天神临死之前有五种征状：衣裳垢腻、头上花萎、身体臭秽、腋下汗出、不乐本
-----------
座（第五个征状或说是“玉女离散”），这就是所谓“天人五衰”，是天神最大的悲哀。帝释是众天神的领袖。
　　“龙”是指龙神。佛经中的龙，和我国传说中的龙大致差不多，不过没有脚，有时大蟒蛇也称为龙。事实上，中
-----------
国人对龙和龙王的观念，一部分从佛经中而来。佛经中有五龙王、七龙王、八龙王等等名称。古印度人对龙很尊敬，认为水中生物以龙的力气最大，陆上生物以象的力气最大，因此对德行崇高的人尊称之为“龙象”，如“西来龙象
-----------


为了能够训练，我们需要一个 `(input, label)` 对的数据集，`input` 和 `label` 都是序列，在每一步输入是当前的字，输出是下一个字。

下面的函数就是实现上面的功能，把每个序列当成是 input，复制，在每一步偏移并对齐文本。

In [32]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [34]:
split_input_target(list("我是北京人"))

(['我', '是', '北', '京'], ['是', '北', '京', '人'])

In [35]:
dataset = sequences.map(split_input_target)

In [43]:
for input_example, target_example in dataset.take(1):
    print("Input :", str(text_from_ids(input_example).numpy(), encoding='utf-8'))
    print("Target:", str(text_from_ids(target_example).numpy(), encoding='utf-8'))

Input : ﻿释名
　　“天龙八部”这名词出于佛经。许多大乘佛经叙述佛陀向诸菩萨、比丘等说法时，常有天龙八部参与听法。如《法华经·提婆达多品》：“天龙八部、人与非人，皆遥见彼龙女成佛”。“非人”是形貌似人而实际不
Target: 释名
　　“天龙八部”这名词出于佛经。许多大乘佛经叙述佛陀向诸菩萨、比丘等说法时，常有天龙八部参与听法。如《法华经·提婆达多品》：“天龙八部、人与非人，皆遥见彼龙女成佛”。“非人”是形貌似人而实际不是


### 创建训练使用的 batches

我们可以使用 `tf.data` 把文本分割成可被管理的序列。但是在放到模型里之前，我们需要把它 shuffle 并且打包成 batches。

In [44]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 构建模型

接下来，我们要使用 `keras.Model` 子类构建模型。 (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

这个模型有三层：

* `tf.keras.layers.Embedding`: 输入层. 一个训练好的 lookup table 能够把每个 character-ID 映射成一个 `embedding_dim` 大小;
* `tf.keras.layers.GRU`: 一个大小为 `units=rnn_units` 的 GRU 结构 (这里也可以使用 LSTM )
* `tf.keras.layers.Dense`: 输出层, 带有 `vocab_size` 大小的输出层. 它会对字典中的每一个字输出一个。这些就是模型对于每个字的 log-likelihood 。

In [45]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Dimensionality of the output space.
rnn_units = 1024

In [46]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [47]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

对于每个字，模型查找 embedding，每个时间步使用 GRU 作为输入，并且使用 Dense 层生成 logics 用来预测下个字的 log-likelihood。

![A drawing of the data passing through the model](http://aimaksen.bslience.cn/text_generation_training.png)

Note: 为了训练，这里也可以使用 `keras.Sequential` 模型. 为了生成文本，我们需要管理 RNN 的内部状态。 但是，使用预置的方式(upfront)会比放在后面再去重新编排模型结构更加的简单。更多的细节请参考 [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## 试用模型

现在尝试的去运行模型，看看是否和预期的一样。

首先，检查一下输出的 shape:

In [48]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 4269) # (batch_size, sequence_length, vocab_size)


在上面的例子中，序列的长度是 `100`，但是模型可以被跑在任何长度的输入上。

In [49]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1092864   
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  4375725   
Total params: 9,406,893
Trainable params: 9,406,893
Non-trainable params: 0
_________________________________________________________________


为了从模型中获取实际的生成文本，我们需要从输出的分布中进行采样，获取真实的字的分片。这个分布是通过 logits 定义的。

注意: 这里从分布中 _sample_ 是非常重要的，我们可以使用分布的 _argmax_ 很容易的从模型中的获取输出。

把这个方式试验在在第一个样本上：

In [51]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

这会给我们，每个时间步，对于下个字的序号的预测:

In [52]:
sampled_indices

array([3840, 2454, 1764, 1512, 2000, 3075, 1812, 2689,   14,  536,   83,
       1547, 3771, 1140,  649, 1183, 3015,  695, 3501, 2684, 4155, 2302,
       2302, 2353, 3014, 2844, 1861, 1337, 2405, 2373,  584, 1867, 2360,
         77, 2576, 2799, 3978,  854, 1829, 3399, 4005, 1249, 2273, 2854,
       2444, 1144,  767, 3463, 1773, 2463,  760, 1400,  129, 2538, 1035,
        766, 1108,   40, 1702,  552, 3323, 3405, 3709, 3897, 3145,  222,
       2785, 1748, 1607, 1734, 3133,  852, 2323,  382, 1097, 4047, 1786,
       1976,  244, 3561,  378,  412, 3839, 2538, 3658, 3683,  327, 2051,
       2110,   91, 2669, 3271, 3851,  818, 1948,  763, 4244,  721, 3539,
       1705])

把这些转化成可读的文本：（此时模型还没有被训练）

In [56]:
print("Input:\n", str(text_from_ids(input_example_batch[0]).numpy(), encoding='utf-8'))
print()
print("Next Char Predictions:\n", str(text_from_ids(sampled_indices).numpy(), encoding='utf-8'))

Input:
 有些卷舌之音，咬字不正，就像是外国人初学中土言语一般。
　　阿朱见少女活泼天真，笑道：“你才长得俊呢，我更加喜欢你！”阿朱久在姑苏，这时说的是中州官话，语音柔媚，可也不甚准确。
　　那渔人本要发怒，见

Next Char Predictions:
 酬玷斤房楚胫是磁C压丝抗通山咂巢耕哲诳碜骏火火煞耐簸朔忡犹爱号朦熄世皙笑陈垒景裘隶度漓粝玉岌嗥设施球嗓恼争痒孤嗤尖t撅叉蛟裳辖银舍余竟散挛放自垂炷冻寻靴既梳侦责决分酪痒踵车兀歪沈中砧蔚采囱格嗜黝啄谭撒


## 训练模型

这时候，我们可以把这个问题当成一个标准的分类的问题。给定之前的 RNN 状态和这个时间步的输入，预测下一个字的类别。

### 指定一个优化器和一个损失函数

标准的 `tf.keras.losses.sparse_categorical_crossentropy` 损失函数可以在这个模型中生效。

因为我们的模型返回的是 logits，所以我们需要设置 `from_logits` 标记。

In [57]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [58]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 4269)  # (batch_size, sequence_length, vocab_size)
Mean loss:         8.358918


一个刚被初始化的模型，还是一个非常不准确的状态，所有的输出的 logits 应该都是比较相似的大小。为了去检查，现在的 mean loss 的指数应该比较接近于词典的大小。如果比这个值大的话，说明这个模型的初始化的并不合理：

In [59]:
tf.exp(mean_loss).numpy()

4268.075

使用 `tf.keras.Model.compile` 配置训练过程，使用 `tf.keras.optimizers.Adam` 默认参数和损失函数。

In [60]:
model.compile(optimizer='adam', loss=loss)

### 配置 checkpoints

使用 `tf.keras.callbacks.ModelCheckpoint` 确保 checkpoints 能够在训练过程中保存:

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 执行训练

In [62]:
EPOCHS = 20

In [63]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
197/197 [==============================] - 173s 869ms/step - loss: 5.7837
Epoch 2/20
197/197 [==============================] - 173s 873ms/step - loss: 4.5910
Epoch 3/20
197/197 [==============================] - 173s 875ms/step - loss: 4.1563
Epoch 4/20
197/197 [==============================] - 170s 857ms/step - loss: 3.8844
Epoch 5/20
197/197 [==============================] - 172s 865ms/step - loss: 3.6819
Epoch 6/20
197/197 [==============================] - 171s 863ms/step - loss: 3.5123
Epoch 7/20
197/197 [==============================] - 170s 859ms/step - loss: 3.3598
Epoch 8/20
197/197 [==============================] - 170s 857ms/step - loss: 3.2177
Epoch 9/20
197/197 [==============================] - 173s 871ms/step - loss: 3.0808
Epoch 10/20
197/197 [==============================] - 173s 871ms/step - loss: 2.9441
Epoch 11/20
197/197 [==============================] - 173s 871ms/step - loss: 2.8101
Epoch 12/20
197/197 [==============================] - 173s 872

## 生成文本

最简单的生成文本的方式就是把模型放到一个 loop 中，跟踪模型的内部的状态。

![To generate text the model's output is fed back to the input](http://aimaksen.bslience.cn/text_generation_training.png)

每个时间步我们会调用模型并且传给模型一个文本和一个内部变量，模型会返回一个预测值和一个新的状态，不断地重复这个过程，生成文本。

下面的是一个 step 的预测：

In [67]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [68]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

把上面的过程放到一个 loop 中来生成文本。我们会发现，模型已经知道什么时候创建一个章节，模型金庸写作的词典。由于训练的 epochs 还比较的小，还不能够生成连贯的句子。

In [69]:
start = time.time()
states = None
next_char = tf.constant(['乔峰带阿朱回到北方，乔峰对她说："我们两人永远留在这里！"'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

乔峰带阿朱回到北方，乔峰对她说："我们两人永远留在这里！"行这里安慰。”鲍千灵道：“多谢乔帮主，哼，向来请下罪名，考乔峰自行，咱们大宋挺兵万家，能有敌眼？”
　　三名老前躬身说道：“巴兄哥，在下这‘生乔’，箫某行侠和众兄之行，可一等相差。当真有甚盼报之乐？”随手将信物长老杂启开口，在守口止步走，说道：“咱们是少林寺方丈玄慈、师伯、玄寂、二弟、三弟，大理国三十三绝，因缘在此后。表哥于丐帮诸兄弟身上多伤的探子，明日三晚六方宫奉。”又有“恭敬”两个小字，便不回公主带同重向。萧峰又羞服侍女，心下意思，不禁又在半空里苦挨腾腾般嚷了起来。
　　暗暗运忆小舟从青城派走了，在少林寺僧人数十三个同时辰中取入的帛轴，不与从幻成中取将回来。
　　玄不见眼见乌老大续道还说，本寺微微曾谈，一束手中找脱藏经等，杀得“先先”也不下去了。一人催道：“止观禅寺开阵，果然乌老大冲施之上，给两人的杖迹把进，掌心手中摇了七八七十来的。老衲远开，请取客气。”他大喜之下，急忙解跃，出气止一张牙，只转过身来，阿碧见鸠摩智口中说什么也完，鸠摩智便须分批“凌波微步”六字，至中没半点头绪，人人均是大伤心，幸亏一篑，沉点向众位东望，说道：“上前出来！只要回家告退，请稍加休息，然回到兴州身边。”
　　段誉心道：“先前愁苦内息，将这三招‘一字半能再算错了，只怕这株山堂也须小弄眼算之上，也有片刻隐隐，再到一个‘龙’字十大破块，就石上磨‘写字，乃是激动无端。只怕性命已不易救，我方得太过神通了这般，在下不可妄了出去。”
　　忽听鸠摩智道：“慕容公子和四大两人都是知道，慕容氏是这件事的，便不可在地底写些什么武功秘势。”段誉听包不同言语相触，便欲觅地不往，站不起身。
　　王语嫣一直不发，心知去伯父、鸠摩智却也在陆地输学，但听慕容复说亲，一烦恼便割他绑缚，不禁年轻感盛，心中霎时间便想：“她造福于我，杀无杀少，从此友有几位大难，在下此刻了不少功劳宫间的那位恩师，王姑娘那个年轻姑娘，是个家慕容公子，想念经动情景，不真心意气躁，只盼能无法不救你一家吧。这……这老义武功比在‘颠倒货色”是以用底极高之若，若能猜得他心生灵念，自由自能。
　　次日段誉和鸠摩智陪过去年，却再也不以为意，实不知他应选吉少。否则从小王府中险无归险，大有十六步而收。段誉记得既是牟尼堂，听说群豪有个多世的机停了，若不是出其喜意，心想通皇太叔这部位如奉伯父性命，只因虚竹很亲自

最简单的提高结果的方式就是训练更多的 epochs (试试 `EPOCHS = 30`).

你还可以使用一个不同的开始的句子做实验，或者尝试增加 RNN 的层数提高模型的准确率，或者是调整 temperature 参数生成一些具有随机性的预测。

如果你想模型生成文本更快，最简单的方式就是你可以使用 batch 的方式生成文本。下面的例子就是给定 5 个输入，同时进行生成。

In [74]:
start = time.time()
states = None
next_char = tf.constant(['乔峰', '阿朱', '段誉', '丐帮', '虚竹'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
for res in result:
    print(str(res.numpy(), encoding='utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

乔峰去了，语气凄凉，增缘相去，陈长老勉强袖言淫动，自己可知便杀了义父、愤怒之仇，都想从当汪帮主朗声说道：“吴长老，乔贵帮甚敬重宽，你也不必大错过大，最不是本领的不老是，我又生兴他也好了？又是江湖之上，还道要抢援的丐帮总是大理国，转过了一个觉，又何必惹人人欺相之身？”
　　他说耶律洪基一箭将出来推对方位，说道：“押三军之路向前，冲着各路快追。”耶律洪基笑道：“遵命大宋官兵下属，当年和气与同东、共领大方，都是臣活人奴才好朋友。”萧峰道：“但那辽军的家伙只要出一个千百人，但皇帝要否换烧，蜂皮肉丝不雄多，就是有十二官的首领。辽兵的千刀万剐，斩草干部要冲木桥，他还活得一下便逃了。”萧峰道：“我自尽了几百招的，只盼收弟。在下三年有一之多，但后来干扰褚万里，明天一救，三天一死，岂不干了。”乔峰道：“遮掩藏书，头一战百余万里没有下足。”东首席上数百个十个响，打入东河的中严，格格的笑道：“皇上鸿目神乎，要废我日后兴。卒官元雄年分西，拜见公子，请出来迎打。”
　　正是前朝便开，每年一名山东泰山已走入二十房里，有饭的朱香、执法僧意令邻林有余人拜了下去，赶后闯下。人人均感不忿，心想：“苦命钻研，迷满之别。其实与太姻重重，临无之间，他也不想必由自恃读经文学佛法，最要先生罪过经书，那时不便枯荣。本来师父都藏在中间，若能为虚竹这点意，倘若将他喝了，这筋斗骨断，定是积贮败辽，回到家后便即一举；以皇太后三年食物以牛竭为粮留守戒，又说何人故强行事，众人就此必小。其时敌国千里，众无常往。只一口推投，连牛派也不杀戒。武僧在临头之外，也属太皇太后两派的好处安。萧君王恩政之辞，我想必将《易筋经》为难平土的首山，取他驱回宫去接安。”段誉拱手道：“那时在卫辉定平要告知。”萧峰又道：“洪基，这么大胆，富贵数千百人，只要再多数十人赏，以免隐隐以为报明。
　　渐行如此，每一百棍都已尽集百川，比之当更繁复，不多点力，便是两条的鼻子方远。萧峰早已瞧得干不净的丑八怪，寻思：“这铁杖流为真婆。我乃皇太后太皇太后，只怕穆贵妃又有又恐怕不尽数号令我要害死父母母恶，为父是杀我妻子，而死粉身谷时时轻柔收气，他便想到正面逃走。嗯，好酒么劝你还没什么君子难劝，他们离长之后，再迟声扰扰，因此哭扰煎养，也不能将来喝酒吃喝水吧。”耶律洪基笑道：“好不但随便送几位大爷锋头，我自今端起了，若不立酒，你便是大祸之望，别让敌人趁机逃回南本，说我辽国不怎

## 导出生成器

这个 single-step 模型可以被轻松的 [saved and restored](https://www.tensorflow.org/guide/saved_model), 允许你在任意位置使用`tf.saved_model`

In [75]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [76]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:，一眨眼间时到火柱，还差着是真正的笑。神仙姊姊可笑不出的光芒，说道：“姑娘，你的险狠会有好东西道一直，再来找个报了。”游坦之道：“姑娘对尊不对容貌是美女，是美丑脸，什么兜也？”
　　只见乌老大捕生生内


## 提高:自定义训练

上面的训练方式比较的简单，不能够给你充分的控制。

我们可以使用 teacher-forcing 来阻止不好的预测生成又重新的输入到模型中，那么模型将永远也不会覆盖掉这些错误了。

所以，现在我们要看看怎么实现这样一个训练的循环。 这个例子给了一个很好的开始，比如说，你想要实现 _curriculum  learning_ 去稳定模型 open-loop 输出。

自定义训练过程里最终要的部分就是自定义一个 step 的计算过程。

使用 `tf.GradientTape` 去跟踪 gradients. 你可以从这里学到更多 [eager execution guide](https://www.tensorflow.org/guide/eager).

基本的过程如下：

1. 执行魔性并且使用 `tf.GradientTape` 计算损失
2. 计算更新，并且使用 optimizer 应用到模型中。

In [77]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {'loss': loss}

上面的 `train_step` 实现遵循 [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). 这个是可选的，但是它可以让你改变你的模型的训练行为，并且依然使用 keras' `Model.compile` 和 `Model.fit` 方法.

In [78]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [79]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [80]:
model.fit(dataset, epochs=1)

197/197 [==============================] - 169s 846ms/step - loss: 5.7795


或者你需要更复杂的控制，你可以写一个完全自定义的训练的 loop：

In [81]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.9512
Epoch 1 Batch 50 Loss 4.7906
Epoch 1 Batch 100 Loss 4.6118
Epoch 1 Batch 150 Loss 4.4033

Epoch 1 Loss: 4.5912
Time taken for 1 epoch 167.37 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 4.2720
Epoch 2 Batch 50 Loss 4.2705
Epoch 2 Batch 100 Loss 4.1763
Epoch 2 Batch 150 Loss 4.0766

Epoch 2 Loss: 4.1649
Time taken for 1 epoch 167.73 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 3.8965
Epoch 3 Batch 50 Loss 3.8972
Epoch 3 Batch 100 Loss 3.9550
Epoch 3 Batch 150 Loss 3.7996

Epoch 3 Loss: 3.8942
Time taken for 1 epoch 167.61 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 3.7211
Epoch 4 Batch 50 Loss 3.7356
Epoch 4 Batch 100 Loss 3.6499
Epoch 4 Batch 150 Loss 3.7698

Epoch 4 Loss: 3.6894
Time taken for 1 epoch 168.13 sec
_________________________________________________________________